# **Q-Learning**

[Gym OpenAI] Nesse tipo de aprendizagem, existe um ambiente que possui um objetivo de otimização e existe um agente neste ambiente que varia estados com base em recompensas (positivas ou negativas). Se ele se aproximar do objetivo, recebe uma recompensa positiva e se afastar do objetivo, a recompensa é negativa.<br><br>
**táxi**<br>
Movimentos e Recompensas:
1. Cima;
2. Baixo;
3. Frente;
4. Trás;
5. Pegar passageiro;
6. Deixar passageiro.
* -10: Buscar o passageiro no lugar errado;
* -10: Deixar o passageiro no lugar errado;
* -1: Andar no ambiente (living penalty);
* +20: Buscar e deixar o passageiro no lugar certo.<br>

**Equação de Bellman**:
$$V(s)=max_a \left( R(s,a)+\gamma V(s') \right)$$
Nesta equação, cada $V(s)$ é um estado (bloquinho) que fornece uma recomensa $R(s,a)$ com $a$ sendo a ação e $s$ o novo estado e adicionamos ao $V(s')$ que se refere ao próximo estado, somado de um parâmetro $\gamma$ (valor de desconto, ex.: 0.9).<br><br>
**Markov Decision Processo (MDP)**:<br>
Refere-se às opções de operar sob um cenário de Exploitation determinístico em que o agente sempre irá para  amelhor posição do ambiente ou de Exploration não determinístico em que há uma maior probebilidade de ele seguir para a melhor posição.<br><br>
**Q-Learning**<br>
É um algoritmo que consiste em uma fórmula matemária para decidir a melhor posição (o agente segue para o valor de maior posição $Q(s_0,\,a_n)$ em que $Q$ pode ser entendido como qualidade, que ocorre antes ao valor de $V(s)$ de Bellman).<br><br>
**Diferença Temporal**:<br>
Todas as vezes que o agente interage com o ambiente, o algoritmo executa um episódio (rodada completa de execução) e retorna um valor que indicarão a qualidade de cada uma das ações.
$$Q_t(s,a)=Q_{t-1}(s,a)+\alpha \left( R(s,a)+\gamma V(s') \right)$$
O parâmetro $\alpha$ é a taxa de aprendizagem (learning rate) que indica o quão rápido o algoritmo aprenderá. Outra forma de escrever esta equação é a seguinte:
$$Q_t(s,a)=Q_{t-1}(s,a)+\alpha TD_t(a,s)$$

`pip install gym`<br>
`pip install pygame`<br>
`pip install `

In [60]:
import gymnasium as gym
import random

In [61]:
import re

def remove_ansi_escape_sequences(text):
    ansi_escape = re.compile(r'(?:\x1B[@-_][0-?]*[ -/]*[@-~])')
    return ansi_escape.sub('', text)

env = gym.make('Taxi-v3', render_mode="ansi")
env.reset()
output = env.render()
clean_output = remove_ansi_escape_sequences(output)
print(clean_output)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [62]:
# 0 = baixo; 1 = north; 2 = east; 3 = west; 4 = pickup; 5 = dropoff
print(env.action_space)

Discrete(6)


In [63]:
# 4 destinations
print(env.observation_space)

Discrete(500)


In [64]:
len(env.P);

/Users/gustavomeira/.pyenv/versions/3.12.0/lib/python3.12/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.P to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.P` for environment variables or `env.get_wrapper_attr('P')` that will search the reminding wrappers.
  logger.warn(


In [65]:
display(env.P[484]);

{0: [(1.0, 484, -1, False)],
 1: [(1.0, 384, -1, False)],
 2: [(1.0, 484, -1, False)],
 3: [(1.0, 464, -1, False)],
 4: [(1.0, 484, -10, False)],
 5: [(1.0, 484, -10, False)]}

## Treinamento

In [66]:
import numpy as np

q_table = np.zeros([env.observation_space.n, env.action_space.n])
display(q_table)

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [69]:
%%time
from IPython.display import clear_output

alpha = 0.1
gamma = 0.6
epsilon = 0.1

for i in range(10000):
    estado = env.reset()

    penalidades, recompensa = 0, 0
    done = False
    while not done:
        if random.uniform(0,1) < epsilon:
            # Exploration
            action = env.action_space.sample()
        else:
            # Exploitation
            action = np.argmax(q_table[estado])

        proximo_estado, recompensa, done, info = env.step(action)

        q_antigo = q_table[estado, action]
        proximo_maximo = np.max(q_table[(proximo_estado),:])

        q_novo = (1-alpha)*q_antigo +alpha*(recompensa + gamma*proximo_maximo)
        q_table[estado,action] = q_novo

        if recompensa == -10:
            penalidades += 1

        estado = proximo_estado

    if i%100 == 0:
        clear_output(wait=True)
        print('Epsilon =',i)

print('Concluído')

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices